In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import sklearn.datasets
import sklearn.model_selection
from sklearn.preprocessing import OneHotEncoder

# import mnist digits dataset
mnist = tf.keras.datasets.mnist # Object of the MNIST dataset
(x_train, y_train),(x_test, y_test) = mnist.load_data()

2023-04-30 17:32:08.262967: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-30 17:32:08.316089: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-30 17:32:08.316983: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-30 17:32:09.641051: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# DATA TRUNCATION FOR TEST
x_train = x_train[:100]
y_train = y_train[:100]
x_test = x_test[:100]
y_test = y_test[:100]

In [3]:
# Normalize the train dataset
x_train = tf.keras.utils.normalize(x_train, axis=1)
# Normalize the test dataset
x_test = tf.keras.utils.normalize(x_test, axis=1)

# Hilbert Curve translation

In [4]:
# Hilbert path generator
# ALL CREDIT TO: https://github.com/jakubcerveny/gilbert/blob/master/gilbert2d.py
# ALL CREDIT TO @jakubcerveny on GitHub

def sgn(x):
    return -1 if x < 0 else (1 if x > 0 else 0)

def generate_path(x,y,ax,ay,bx,by):
    w = abs(ax + ay)
    h = abs(bx + by)

    (dax, day) = (sgn(ax), sgn(ay)) # unit major direction
    (dbx, dby) = (sgn(bx), sgn(by)) # unit orthogonal direction

    if h == 1:
        # trivial row fill
        for i in range(0, w):
            yield(x, y)
            (x, y) = (x + dax, y + day)
        return

    if w == 1:
        # trivial column fill
        for i in range(0, h):
            yield(x, y)
            (x, y) = (x + dbx, y + dby)
        return

    (ax2, ay2) = (ax//2, ay//2)
    (bx2, by2) = (bx//2, by//2)

    w2 = abs(ax2 + ay2)
    h2 = abs(bx2 + by2)

    if 2*w > 3*h:
        if (w2 % 2) and (w > 2):
            # prefer even steps
            (ax2, ay2) = (ax2 + dax, ay2 + day)

        # long case: split in two parts only
        yield from generate_path(x, y, ax2, ay2, bx, by)
        yield from generate_path(x+ax2, y+ay2, ax-ax2, ay-ay2, bx, by)

    else:
        if (h2 % 2) and (h > 2):
            # prefer even steps
            (bx2, by2) = (bx2 + dbx, by2 + dby)

        # standard case: one step up, one long horizontal, one step down
        yield from generate_path(x, y, bx2, by2, ax2, ay2)
        yield from generate_path(x+bx2, y+by2, ax, ay, bx-bx2, by-by2)
        yield from generate_path(x+(ax-dax)+(bx2-dbx), y+(ay-day)+(by2-dby),
                              -bx2, -by2, -(ax-ax2), -(ay-ay2))

def hilbert_path(n):
    yield from generate_path(0,0,n,0,0,n)


In [5]:
def dim_reduction(image):
    # Translate 2D image into 1D vector using Hilbert curve
    # image: 2D numpy array (28x28) or (nxn)
    # return: 1D numpy array
    width = image.shape[0]
    
    # Generate Hilbert curve path
    path = hilbert_path(width)
    # 1D vector widthxwidth
    vector = np.zeros(width*width)
    # Fill vector with image values
    for i, (x, y) in enumerate(path):
        vector[i] = image[x][y]
    return vector


In [32]:
# # Basic test
# print(np.array([[0,0,.25,.25],[0,0,.25,.25],[.75,.75,1,1],[.75,.75,1,1]]))
# print(dim_reduction(np.array([[0,0,.25,.25],[0,0,.25,.25],[.75,.75,1,1],[.75,.75,1,1]])))

### Translating MNIST

Note: This takes a long time!

In [6]:
x_train = np.array([dim_reduction(x) for x in x_train])
x_test = np.array([dim_reduction(x) for x in x_test])

### Converting to time series

In [7]:
train_width = 28*28*x_train.shape[0]
test_width = 28*28*x_test.shape[0]
x_train = x_train.flatten()
x_test = x_test.flatten()

In [8]:
y_train = np.array([[y for _ in range(28*28)] for y in y_train]).flatten()
y_test = np.array([[y for _ in range(28*28)] for y in y_test]).flatten()

In [9]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

enc = enc.fit(y_train.reshape(-1,1))

y_train = enc.transform(y_train.reshape(-1,1))
y_test = enc.transform(y_test.reshape(-1,1))

# Recurrent Neural Network

# LSTM RNN

In [10]:
print(x_train.shape)
x_train = x_train.reshape(-1, 1, 1)
x_test = x_test.reshape(-1, 1, 1)
print(x_train.shape)


(78400,)
(78400, 1, 1)


In [39]:
model = tf.keras.models.Sequential()
model.add(
    keras.layers.Bidirectional(
      keras.layers.LSTM(
          units=128,
          input_shape=[x_train.shape[1], x_train.shape[2]]
          
      )
    )
)
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(y_train.shape[1], activation='softmax'))

model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['acc']
)

In [40]:
history = model.fit(
    x_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    shuffle=False
)

Epoch 1/20


2023-04-03 16:31:58.356463: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-03 16:31:58.358842: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-03 16:31:58.359762: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2199/2205 [============================>.] - ETA: 0s - loss: 2.2275 - acc: 0.1954

2023-04-03 16:32:11.957470: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-03 16:32:11.960009: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-03 16:32:11.961348: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2205/2205 [==============================] - 15s 5ms/step - loss: 2.2281 - acc: 0.1949 - val_loss: 2.3360 - val_acc: 0.0000e+00
Epoch 2/20
2205/2205 [==============================] - 11s 5ms/step - loss: 2.2501 - acc: 0.1960 - val_loss: 2.3776 - val_acc: 0.1000
Epoch 3/20
2205/2205 [==============================] - 11s 5ms/step - loss: 2.2006 - acc: 0.2167 - val_loss: 2.5055 - val_acc: 0.0000e+00
Epoch 4/20
2205/2205 [==============================] - 11s 5ms/step - loss: 2.2016 - acc: 0.1811 - val_loss: 2.4731 - val_acc: 0.1000
Epoch 5/20
2205/2205 [==============================] - 13s 6ms/step - loss: 2.2512 - acc: 0.1533 - val_loss: 2.3723 - val_acc: 0.1000
Epoch 6/20
2205/2205 [==============================] - 10s 5ms/step - loss: 2.1914 - acc: 0.2015 - val_loss: 2.4653 - val_acc: 0.1000
Epoch 7/20
2205/2205 [==============================] - 11s 5ms/step - loss: 2.1879 - acc: 0.1860 - val_loss: 2.4169 - val_acc: 0.0000e+00
Epoch 8/20
2205/2205 [==============================] 